In [1]:
!pip install extra_keras_datasets -q

In [2]:
from matplotlib import pyplot as plt
import numpy as np
from numpy.linalg import matrix_power
import random
import math as m
import scipy

from sklearn.metrics.pairwise import rbf_kernel
from extra_keras_datasets import usps
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [3]:
(X_train, y_train), (X_test,y_test) = usps.load_data()
X_train, y_train = shuffle(X_train, y_train)
filter_nine = np.where(y_train == 9)
filter_four = np.where(y_train == 4)
X_train_9 = X_train[filter_nine]
X_train_4 = X_train[filter_four]

INFO:root:Loading dataset = usps


In [4]:
X_train_4_500 = X_train_4[0:500]
X_train_9_500 = X_train_9[0:500]

y_train_4_500 = [-1]*500
y_train_9_500 = [1]*500

X_train_data = np.append(X_train_4_500, X_train_9_500, axis=0)
X_train_data = X_train_data.reshape(-1, 16*16)
y_train_data = y_train_4_500 + y_train_9_500
y_train_data = np.array(y_train_data)

X_train_data.shape, y_train_data.shape

temp = np.zeros((X_train_data.shape[0], X_train_data.shape[1]+1))
temp[:,:-1] = X_train_data
temp[:,-1] = y_train_data
temp.shape

(1000, 257)

In [5]:
W_all = rbf_kernel(X_train_data, )

In [6]:
# print(X_train_9.shape)
# num = random.randint(0, 644)
# plt.imshow(X_train_9[num])

In [7]:
# print(X_train_4.shape)
# num = random.randint(0, 654)
# plt.imshow(X_train_4[num])

In [8]:
X_train_9_5 = X_train_9[0:5]
X_train_4_5 = X_train_4[0:5]
X_train_4_5.shape

(5, 16, 16)

In [9]:
y_train = [1,1,1,1,1,-1,-1,-1,-1,-1]
len(y_train)

10

In [10]:
data_labeled = np.append(X_train_4_5, X_train_9_5, axis=0)
data_labeled.shape

(10, 16, 16)

In [11]:
# num = random.randint(0, 250)
# print(num)
# plt.imshow(X_train[num])
# print(y_train[num])

In [12]:
def GaussianKernel(v1, v2, sigma):
    return scipy.exp(-np.linalg.norm(v1-v2, 2)**2/(2.*sigma**2))

In [13]:
sigma = 5
gamma_ = 1 / (2*(sigma**2))
# gamma_ = 0.47

In [14]:
def RobustGC(W, y_train_prime):
  eta = 0.9
  D = np.zeros ((len(W),len(W)))
  for i in range (len(W)):
    D[i][i]= np.sum(W[i] , axis=0)

  D_inv = np.sqrt(matrix_power(D,-1))
  S = D_inv @ W @ D_inv
  LN = np.eye(len(S)) - S
  V0 = np.zeros(len(D))

  for i in range (len (D)):
    V0[i]= np.sqrt(D[i][i])

  V0_norm = np.linalg.norm(V0, 2)
  V0= V0/V0_norm
  v=np.linalg.eigvals(LN)
  v=sorted(v)
  lambda_1 = v[1]
  gamma_t= eta*lambda_1
  # print(gamma_t)
  
  f = np.linalg.inv((LN/gamma_t)-np.eye(len(LN)))@(y_train_prime - (np.dot(V0,np.dot(V0.transpose(), y_train_prime))))
  pred= np.sign(f)
  pred = np.squeeze(np.asarray(pred))
  # print (pred)

  return f, gamma_t

In [15]:
D_i_all = np.zeros(1000)

In [16]:
def kernel_calc(X_train_data, X_train):
  res = rbf_kernel(X_train_data, X_train ,gamma=gamma_)
  # res = rbf_kernel(X_train_data, X_train ,gamma=0.47)
  
  res_dic = {}
  for i in range(res.shape[0]):
    for j in range(res.shape[1]):
      res_dic[(i,j)] = res[i,j]
  
  return res_dic

In [17]:
# d_i_all = np.zeros(1000)

def d_x (num, kernel_val):
  result=0
  for i in range (len(X_train)):
    # result +=  kernel_val[num , i]
    result += kernel_val[(num, i)]
  return result

def predx (x, num, kernel_val, prev):
  # sigma = 5
  # for i in range (prev , len(X_train)):
  #   d_i_all[num] += kernel_val[num , i]

  seg_1= 1/(1-gamma_t-(1/d_x(num, kernel_val)))
  seg_2= 0
  temp1 = d_x(num, kernel_val)

  all_di_temp = rbf_kernel(X_train, X_train ,gamma=gamma_)
  all_di = np.sum(all_di_temp, axis=1)

  for i in range (len(X_train)):
    seg_2 += (kernel_val[num,i]/np.sqrt(temp1*all_di[i])) * pred[i] 
  result = seg_1 * seg_2
  # return result

  return np.sign(result)
  
# def predx (x, num, kernel_val):
#   return np.sign(f_x(x, num, kernel_val))

In [18]:
prev = 0

In [19]:
for data_num in range(20, 1001, 50):
  y_prime = []
  temp = data_num - 10
  # data = np.array([])
  data_9 = X_train_9[5: int(temp/2)+5]
  data_4 = X_train_4[5: int(temp/2)+5]
  data = np.concatenate([data_labeled, data_4, data_9], axis=0)
  
  y_data = y_train + [0]*temp
  
  X_train, y_data = shuffle(data, y_data)
  X_train = X_train.reshape(-1, 16*16)

  W = rbf_kernel(X_train, gamma=gamma_)
  pred, gamma_t = RobustGC(W, y_data)
  # pred = - pred
  
  kernel_val = kernel_calc(X_train_data, X_train)
  # print(kernel_val[0,0])
  
  for num, d in enumerate(X_train_data):
    t = -predx(d, num, kernel_val, prev)
    y_prime.append(t)
  prev = len(X_train)


  acc = accuracy_score(y_train_data, y_prime, normalize=True)
  print('accuracy= ', acc)
  # # print("|", end='')

accuracy=  0.783
accuracy=  0.859
accuracy=  0.861
accuracy=  0.861
accuracy=  0.852
accuracy=  0.853
accuracy=  0.855
accuracy=  0.859
accuracy=  0.861
accuracy=  0.861
accuracy=  0.861
accuracy=  0.86
accuracy=  0.861
accuracy=  0.859
accuracy=  0.857
accuracy=  0.859
accuracy=  0.858
accuracy=  0.858
accuracy=  0.856
accuracy=  0.86
